In [91]:
import json
import random
import matplotlib.pyplot as plt
import numpy as np
from itertools import chain, repeat
import task_generation.taskset_generation as tg
import uuid
from enum import Enum
from concurrent.futures import ProcessPoolExecutor

In [3]:
class PTask:
    def __init__(self, period, execution_time):
        self.period = period
        self.execution_time = execution_time

    @property
    def utilization(self):
        return self.execution_time / self.period

    def __str__(self):
        return f"({self.period}, {self.execution_time})"

    def __repr__(self):
        return f"({self.period}, {self.execution_time})"


class Task:
    def __init__(self, arrival_time, deadline, execution_time):
        self.id = uuid.uuid4()
        self.arrival_time = arrival_time
        self.deadline = deadline
        self.execution_time = execution_time
        self.start_time = 0
        self.finish_time = 0
        self.waiting_time = 0
        self.response_time = 0
        self.slack_time = 0

    @property
    def utilization(self):
        return self.execution_time / (self.deadline - self.arrival_time)

    def __str__(self):
        return f"({self.arrival_time}, {self.deadline}, {self.execution_time})"

    def __repr__(self):
        return f"({self.id}, {self.arrival_time}, {self.deadline}, {self.execution_time})"

In [4]:
class Scheduler:
    def __init__(self, num_tasks, mutation_rate, crossover_rate, max_iter):
        self.num_tasks = num_tasks
        self.mutation_rate = mutation_rate
        self.population = []
        self.best_fitness = 0
        self.best_chromosome = []

    def initialize(self):
        # code to initialize population with random chromosomes
        pass

    def crossover(self):
        # code to perform crossover between parent chromosomes
        pass

    def mutate(self):
        # code to perform mutation on child chromosomes
        pass

    def fitness(self):
        # code to calculate fitness of each chromosome in the population
        pass

    def selection(self):
        # code to perform selection of fittest chromosomes for next generation
        pass

    def evolve(self, num_generations):
        # code to run GA algorithm for given number of generations
        pass

    def run(self, tasks):
        # code to run the scheduler for given tasks using GA algorithm
        pass


class GA_Scheduler(Scheduler):
    """
    A class representing the GA algorithm for task scheduling.
    """

    def __init__(self, num_tasks, mutation_rate, crossover_rate, max_iter):

        self.pop_size = num_tasks
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.max_iter = max_iter
        super().__init__(num_tasks, mutation_rate, crossover_rate, max_iter)

    def initialize_population(self, num_tasks):
        """
        Initializes the population with random task orders.

        Args:
            num_tasks (int): Number of tasks.

        Returns:
            list: A list of task orders (chromosomes).
        """
        population = []
        for i in range(self.pop_size):
            chromosome = [j for j in range(num_tasks)]
            random.shuffle(chromosome)
            population.append(chromosome)
        return population

    def calculate_fitness(self, population, tasks):
        """
        Calculates the fitness of each chromosome in the population.

        Args:
            population (list): A list of task orders (chromosomes).
            tasks (list): A list of Task objects.

        Returns:
            list: A list of tuples containing the chromosome and its fitness.
        """
        fitness_scores = []
        for chromosome in population:
            full_time, final_time, wait_time, resp_time, slack_time = self.evaluate(chromosome, tasks)
            fitness = 1 / (1 + slack_time)
            fitness_scores.append((chromosome, fitness))
        return fitness_scores

    def evaluate(self, chromosome, tasks):
        """
        Evaluates a chromosome by simulating the execution of the tasks.

        Args:
            chromosome (list): A list representing the order of tasks to be executed.
            tasks (list): A list of Task objects.

        Returns:
            tuple: A tuple containing the full time, final time, wait time, response time, and slack time.
        """
        # TODO: Implement the task execution simulation.
        pass

    def selection(self, fitness_scores):
        """
        Selects two parent chromosomes using tournament selection.

        Args:
            fitness_scores (list): A list of tuples containing the chromosome and its fitness.

        Returns:
            tuple: A tuple containing the two parent chromosomes.
        """
        # TODO: Implement tournament selection.
        pass

    def crossover(self, parent1, parent2):
        """
        Performs crossover between two parent chromosomes.

        Args:
            parent1 (list): The first parent chromosome.
            parent2 (list): The second parent chromosome.

        Returns:
            list: The child chromosome resulting from crossover.
        """
        # TODO: Implement crossover.
        pass

    def mutate(self, chromosome):
        """
        Mutates a chromosome.

        Args:
            chromosome (list): The chromosome to mutate.

        Returns:
            list: The mutated chromosome.
        """
        # TODO: Implement mutation.
        pass

    def evolve(self, population, fitness_scores):
        """
        Evolves the population by selecting parents, performing crossover and mutation, and generating a new population.

        Args:
            population (list): A list of task orders (chromosomes).
            fitness_scores (list): A list of tuples containing the chromosome and its fitness.

        Returns:
            list: A new list of task orders (chromosomes).
        """
        # TODO: Implement evolution.
        pass

    def run(self, tasks):
        """
        Runs the GA
        """
        pass

In [5]:
def generate_tasks(num_tasks):
    # code to generate random tasks with given number of tasks
    pass


def read_tasks_from_file(file_path):
    # code to read tasks from file and create Task objects
    pass


def write_tasks_to_file(tasks):
    # code to write tasks to JSON file
    pass


def write_results_to_file(results):
    # code to write results to JSON file
    pass


def plot_results(num_tasks_list, ga_results, improved_ga_results):
    # code to plot bar chart of average times for different number of tasks for both algorithms
    pass

In [6]:
def generate_ptasks(num_tasks=100, utilization=.5, available_periods=(1, 2, 4, 8, 16)):
    """
    Generates a set of periodic tasks with random periods and costs.
    """
    num_sets = 1
    u = np.array(tg.generate_uunifastdiscard(num_sets, utilization, num_tasks, 'test.csv'))
    p = np.array(tg.generate_random_periods_discrete(num_tasks, num_sets, available_periods))
    tset = np.array(tg.generate_tasksets(u, p, 'tset.csv'), dtype=[('cost', 'float64'), ('period', 'int32')])[0]
    tset = [PTask(tset['period'][i], tset['cost'][i]) for i in range(len(tset))]
    return tset


def generate_tasks(tset):
    """
    Generates a list of tasks from a set of periodic tasks.
    """
    if not tset:
        return []
    hp = np.lcm.reduce([t.period for t in tset])
    tasks = list(chain.from_iterable([
        zip(
            list(range(0, hp, tset[i].period)),
            list(range(tset[i].period, hp + 1, tset[i].period)),
            list(repeat(tset[i].execution_time, hp // tset[i].period))
        ) for i in range(len(tset))
    ]))
    # convert tasks to Task objects
    tasks = [Task(task[0], task[1], task[2]) for task in tasks]
    return tasks

In [7]:
class MAPPING_ALGORITHM(Enum):
    """
    An enum representing the mapping algorithm to use.
    """
    FIRST_FIT_DECREASING = 1
    BEST_FIT_DECREASING = 2
    WORST_FIT_DECREASING = 3
    FIRST_FIT_INCREASING = 4
    BEST_FIT_INCREASING = 5
    WORST_FIT_INCREASING = 6


def map_tasks_to_cores(ptasks, num_cores, mapping_algorithm):
    """
    Maps tasks to cores using the specified mapping algorithm.
    """
    # sort tasks based on utilization based on the mapping algorithm
    if mapping_algorithm.value <= 3:
        ptasks.sort(key=lambda task: task.utilization, reverse=True)
    else:
        ptasks.sort(key=lambda task: task.utilization, reverse=False)

    core_remaining_utilization = [1 / num_cores for _ in range(num_cores)]
    core_tasks = [[] for _ in range(num_cores)]

    # map tasks to cores
    for task in ptasks:
        if mapping_algorithm.value % 3 == 1:
            # first fit
            for i in range(num_cores):
                if core_remaining_utilization[i] >= task.utilization:
                    core_tasks[i].append(task)
                    core_remaining_utilization[i] -= task.utilization
                    break
            else:
                raise Exception('No core has enough remaining utilization to map task.')
        elif mapping_algorithm.value % 3 == 2:
            # best fit
            best_fit = -1
            for i in range(num_cores):
                if core_remaining_utilization[i] >= task.utilization:
                    if best_fit == -1 or core_remaining_utilization[i] < core_remaining_utilization[best_fit]:
                        best_fit = i
            if best_fit != -1:
                core_tasks[best_fit].append(task)
                core_remaining_utilization[best_fit] -= task.utilization
            else:
                raise Exception('No core has enough remaining utilization to map task.')
        else:
            # worst fit
            worst_fit = -1
            for i in range(num_cores):
                if core_remaining_utilization[i] >= task.utilization:
                    if worst_fit == -1 or core_remaining_utilization[i] > core_remaining_utilization[worst_fit]:
                        worst_fit = i
            if worst_fit != -1:
                core_tasks[worst_fit].append(task)
                core_remaining_utilization[worst_fit] -= task.utilization
            else:
                raise Exception('No core has enough remaining utilization to map task.')

    return core_tasks

In [226]:
class Particle:
    def __init__(self):
        self._order = None
        self.best = None
        self.fitness = None
        self.final_time = None
        self.avg_lateness = None
        self.avg_wait_time = None
        self.avg_resp_time = None
        self.avg_slack_time = None

    def set_order(self, order):
        self._order = order
        return self

    def get_order(self):
        return self._order

    def save_if_better(self, particle):
        if self.best is None or (self.best.fitness < particle.fitness and
                                 ((self.best.avg_lateness > 0) or (particle.avg_lateness == 0))):
            self.best = particle.copy()

    def copy(self):
        particle = Particle()
        particle.set_order(self._order.copy())
        particle.fitness = self.fitness
        particle.final_time = self.final_time
        particle.avg_lateness = self.avg_lateness
        particle.avg_wait_time = self.avg_wait_time
        particle.avg_resp_time = self.avg_resp_time
        particle.avg_slack_time = self.avg_slack_time
        return particle


class PSOScheduler:
    def __init__(self, w, c1, c2, tasks):
        self.particles, self.particle_count = None, None
        self.best_keeper = Particle()
        self.tasks = tasks
        self.w, self.c1, self.c2 = w, c1, c2

    def calculate_fitness(self, particle):
        final_time, avg_wait_time, avg_resp_time, avg_lateness, avg_slack_time = self.evaluate(particle.get_order())
        fitness = 1 / (np.exp((avg_lateness*10 + 1)*(avg_resp_time + avg_wait_time)))

        particle.fitness, particle.avg_lateness, particle.avg_wait_time, particle.avg_resp_time, particle.avg_slack_time = fitness, avg_lateness, avg_wait_time, avg_resp_time, avg_slack_time
        particle.save_if_better(particle)

    def evaluate(self, order):
        """
        Evaluates an order by simulating the execution of the tasks.

        Args:
            order (list): A list representing the order of tasks to be executed.
        Returns:
            tuple: A tuple containing the full time, final time, wait time, response time, and slack time.
        """
        final_time = 0
        wait_time = 0
        resp_time = 0
        lateness = 0
        slack_time = 0

        for task_id in order:
            task = self.tasks[task_id]
            task.start_time = max(task.arrival_time, final_time)
            task.finish_time = task.start_time + task.execution_time
            task.waiting_time = task.start_time - task.arrival_time
            task.response_time = task.finish_time - task.arrival_time
            task.lateness = task.finish_time - task.deadline
            task.slack_time = task.start_time - final_time

            final_time = task.finish_time
            wait_time += task.waiting_time
            resp_time += task.response_time
            lateness += max(0, task.lateness)
            slack_time += task.slack_time

        return final_time, wait_time/len(order), resp_time/len(order), lateness/len(order), slack_time/len(order)

    def initiate_swarm(self, particle_count):
        self.particle_count = particle_count
        self.particles = []
        for _ in range(self.particle_count):
            order = list(range(len(self.tasks)))
            random.shuffle(order)
            particle = Particle()
            particle.set_order(order)
            self.calculate_fitness(particle)
            self.best_keeper.save_if_better(particle.best)
            self.particles.append(particle)

    def move_particles(self):
        for particle in self.particles:
            self.local_search(particle)  # M1
            self.path_relinking(particle, self.best_keeper.best.get_order(), self.c2)  # M3
            self.path_relinking(particle, particle.best.get_order(), self.c1)  # M2

    def local_search(self, particle):
        """
        insert some of the tasks with the highest lateness in the first position after its release time
        """
        steps = int(len(self.tasks) / 10 * self.w)

        x = particle.get_order()
        self.evaluate(x) # O(n)
        lateness = [(self.tasks[task_id].lateness, task_id) for task_id in x] # O(n)
        tasks_to_move = [task_id for _, task_id in sorted(lateness, reverse=True)[:steps]] # O(nlogn)
        arrival_times = [(self.tasks[task_id].arrival_time, task_id) for task_id in tasks_to_move]
        arrival_times.sort() # O(nlogn)
        arrival_index = 0
        task_id_to_order = {task_id: i for i, task_id in enumerate(x)}
        for task_id in x:
            if self.tasks[task_id].arrival_time > arrival_times[arrival_index][0]:
                task_to_move = arrival_times[arrival_index][1]
                task_id_to_order[task_to_move] = task_id_to_order[task_id] - .5
                arrival_index += 1
                if arrival_index == len(arrival_times):
                    break
        x = [task_id for task_id, _ in sorted(task_id_to_order.items(), key=lambda x: x[1])]
        particle.set_order(x)
        self.calculate_fitness(particle) # O(n)
        self.best_keeper.save_if_better(particle.best)


    def calculate_distance(self, x, y):
        """
        Calculates the distance between two orders. i.e. the minimum number of swaps required to transform one order into the other.
        """
        x = x.copy()
        x_to_index = {task_id: i for i, task_id in enumerate(x)}
        swap_count = 0
        for i in range(len(x)):
            if x[i] != y[i]:
                swap_count += 1
                j = x_to_index[y[i]]
                x[i], x[j] = x[j], x[i]
                x_to_index[x[i]], x_to_index[x[j]] = x_to_index[x[j]], x_to_index[x[i]]
        return swap_count

    def path_relinking(self, particle, t, c):
        x = particle.get_order()
        max_distance = self.calculate_distance(x, t)
        phi = np.random.uniform(0, 1)
        steps = int((phi + c) / 2 * max_distance)
        x_to_index = {task_id: i for i, task_id in enumerate(x)}

        unequal_indices = [i for i in range(len(x)) if x[i] != t[i]]
        np.random.shuffle(unequal_indices)
        random_indices = unequal_indices[:steps]
        for step in range(steps):
            i = random_indices[step]
            j = x_to_index[t[i]]
            x[i], x[j] = x[j], x[i]
            x_to_index[x[i]], x_to_index[x[j]] = x_to_index[x[j]], x_to_index[x[i]]
        particle.set_order(x)
        self.calculate_fitness(particle)
        self.best_keeper.save_if_better(particle.best)

    def run(self, max_iterations):
        for i in range(max_iterations):
            print(f"iteration {i}")
            self.move_particles()
            final_time, avg_wait_time, avg_resp_time, avg_lateness, avg_slack_time = self.evaluate(self.best_keeper.best.get_order())
            print(f"ftime: {final_time:.4f}, avgwtime: {avg_wait_time:.4f}, avgrtime: {avg_resp_time:.4f}, avgstime: {avg_slack_time:.4f} avgl: {avg_lateness:.4f}")
        return self.best_keeper.best

In [214]:
ptasks = generate_ptasks(100, utilization=.9)
mapped_tasks = map_tasks_to_cores(ptasks, 4, MAPPING_ALGORITHM.WORST_FIT_DECREASING)
tasks = [generate_tasks(tset) for tset in mapped_tasks]

In [178]:
def simulate(tasks):
    pso = PSOScheduler(.9, 0.5, 0.5, tasks)
    pso.initiate_swarm(20)
    return pso.run(1000)

In [227]:
simulate(tasks[0])

/tmp/ipykernel_29021/2604278999.py:45: RuntimeWarning: overflow encountered in exp
  fitness = 1 / (np.exp((avg_lateness*10 + 1)*(avg_resp_time + avg_wait_time)))


iteration 0
ftime: 18.2852, avgwtime: 9.3351, avgrtime: 9.3572, avgstime: 0.0901 avgl: 7.0779
iteration 1
ftime: 18.2852, avgwtime: 9.3351, avgrtime: 9.3572, avgstime: 0.0901 avgl: 7.0779
iteration 2
ftime: 18.2852, avgwtime: 9.3351, avgrtime: 9.3572, avgstime: 0.0901 avgl: 7.0779
iteration 3
ftime: 18.2852, avgwtime: 9.3351, avgrtime: 9.3572, avgstime: 0.0901 avgl: 7.0779
iteration 4
ftime: 18.2852, avgwtime: 9.3351, avgrtime: 9.3572, avgstime: 0.0901 avgl: 7.0779
iteration 5
ftime: 18.2852, avgwtime: 9.3351, avgrtime: 9.3572, avgstime: 0.0901 avgl: 7.0779
iteration 6
ftime: 18.2852, avgwtime: 9.3351, avgrtime: 9.3572, avgstime: 0.0901 avgl: 7.0779
iteration 7
ftime: 18.2852, avgwtime: 9.3351, avgrtime: 9.3572, avgstime: 0.0901 avgl: 7.0779
iteration 8
ftime: 18.2852, avgwtime: 9.3351, avgrtime: 9.3572, avgstime: 0.0901 avgl: 7.0779
iteration 9
ftime: 18.2852, avgwtime: 9.3351, avgrtime: 9.3572, avgstime: 0.0901 avgl: 7.0779
iteration 10
ftime: 18.2852, avgwtime: 9.3351, avgrtime: 9.3

KeyboardInterrupt: 

In [117]:
# run pso for each core on a separate process using processPoolExecutor
process_pool = ProcessPoolExecutor(max_workers=4)
futures = [process_pool.submit(simulate, tasks[i]) for i in range(4)]
results = [future.result() for future in futures]

iteration 0
iteration 0
iteration 0
iteration 0
final time: 16.559399000000003, avg wait time: 7.255404030769228, avg resp time: 7.283227315384612, avg lateness: 4.555660261538465
iteration 1
final time: 16.426778, avg wait time: 7.22316703174603, avg resp time: 7.250658952380951, avg lateness: 4.453998539682539
iteration 1
final time: 16.81428800000001, avg wait time: 7.0099314388489224, avg resp time: 7.038535338129499, avg lateness: 4.520928496402878
iteration 1
final time: 15.770941000000006, avg wait time: 6.809828812903226, avg resp time: 6.831395058064516, avg lateness: 4.725935625806452
iteration 1
final time: 16.18609, avg wait time: 6.6106935692307704, avg resp time: 6.638516853846156, avg lateness: 4.010114969230773
iteration 2
final time: 16.095385000000004, avg wait time: 6.542754055555558, avg resp time: 6.5702459761904795, avg lateness: 3.848926650793652
iteration 2
final time: 16.191521000000016, avg wait time: 6.37738043165468, avg resp time: 6.405984330935255, avg lat

KeyboardInterrupt: 

In [100]:
for i, result in enumerate(results):
    print(PSOScheduler(0,0,0, tasks[i]).evaluate(result[0]))

(15.161382999999999, 0.34822365161290364, 0.3697898967741939, 0.0005767548387096757)
(15.034241, 0.36195859523809515, 0.3894505158730158, 8.709523809523665e-05)
(15.048488, 0.6020932692307691, 0.629916553846154, 0.0005399384615384652)
(15.112958, 0.5347380431654678, 0.5633419424460433, 0.0)
